In [42]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,KFold
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder


In [43]:
df_train = pd.read_csv('../../../datasets/parte2/treino/dataset_prepared.csv', na_filter= False)
df_test = pd.read_csv('../../../datasets/parte2/teste/dataset_prepared.csv')

In [44]:
lb_make = LabelEncoder()
df_train['injection_encoded'] = lb_make.fit_transform(df_train['injection'])
df_train.drop('injection',inplace= True,axis=1)

X = df_train.drop('injection_encoded', axis=1)
y = df_train['injection_encoded']

In [45]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [46]:
%pip install -upgrade keras


Usage:   
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] [-e] <vcs project url> ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] [-e] <local project path> ...
  /Users/goncalobraga/miniconda3/envs/DAA/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [47]:
 # %pip install scikeras[tensorflow]

In [48]:
def build_model(activation = 'relu',learning_rate =0.01):
    model = Sequential()
    model.add(Dense(16,input_dim=21, activation = activation ))
    model.add(Dense(8,activation=activation))
    model.add(Dense(1,activation=activation)) # output 
    #Compile the model
    model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.optimizers.Adam(learning_rate),
    metrics = ['accuracy'])
    return model 

In [49]:
model = build_model()
model.summary()


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 16)                352       


                                                                 
 dense_43 (Dense)            (None, 8)                 136       
                                                                 
 dense_44 (Dense)            (None, 1)                 9         
                                                                 
Total params: 497
Trainable params: 497
Non-trainable params: 0
_________________________________________________________________


In [50]:
optimizer = ['Adam', 'SGD','RMSProp']
param_grid = dict(optimizer=optimizer)

In [51]:
kf = KFold(n_splits=2, shuffle=True, random_state=2023)

In [52]:
model = KerasClassifier(build_fn=build_model, epochs=1, batch_size=32, verbose=0)


In [53]:
grid_search = GridSearchCV(estimator = model, param_grid= param_grid, cv = kf, refit='True',verbose = 1 )

In [54]:
print(X)
print(y)

        temp  feels_like  temp_min  temp_max  pressure  humidity  wind_speed  \
0      13.97       13.54     11.45     14.04      1027        81        1.96   
1      13.48       13.02     13.43     13.90      1028        82        1.83   
2      12.93       12.47     12.72     13.43      1027        84        1.96   
3      12.61       12.15     10.34     12.93      1027        85        1.85   
4      12.61       12.17      9.79     12.93      1027        86        1.83   
...      ...         ...       ...       ...       ...       ...         ...   
11011  15.70       15.57     15.57     15.72      1018        86        5.56   
11012  15.55       15.43     12.72     15.70      1018        87        4.47   
11013  13.45       13.28     12.23     13.99      1019        93        3.29   
11014  12.93       12.73     12.23     13.43      1019        94        1.49   
11015  12.93       12.76     12.23     13.43      1019        95        1.93   

       clouds_all  hour  normal  ...  s

In [55]:
grid_search.fit(X,y)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2023-12-18 19:35:01.630065: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 4 which is outside the valid range of [0, 1).  Label values: 3 3 3 4 0 3 3 0 3 3 4 0 4 3 3 3 3 3 3 3 3 3 4 3 2 2 3 4 0 3 4 3
/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2023-12-18 19:35:02.012413: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 4 which is 

ValueError: 
All the 6 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/Users/goncalobraga/miniconda3/envs/DAA/lib/python3.10/site-packages/scikeras/wrappers.py", line 928, in _fit
    self._fit_keras_

In [ ]:
best_mlp_model = grid_search.best_estimator_
print(best_mlp_model)

: 

In [ ]:
best_mlp_model.fit(X,y,epochs = 20, validation_data = (X,y), verbose = 1)

: 

In [ ]:
predictions = best_mlp_model.predict(df_test)

: 

In [ ]:
df_predicoes = pd.DataFrame({'Result': predictions})
df_predicoes['RowId'] = range(1, len(predictions) + 1)
df_predicoes = df_predicoes[['RowId', 'Result']]

df_predicoes.to_csv('../../../datasets/parte2/teste/neuralNetwork.csv', index=False)

: 